In [2]:
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy2
import re

import csv
import requests as rq
import time

from sklearn.feature_extraction.text import CountVectorizer

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore")

In [ ]:
class feature_generation(object):
    """ этот класс содержит методы для создания новых признаков
        и сопутствующие им методы """
    
    def __init__(self, df_movies):
        """ конструктор класса """
        self.movies = df_movies    #исходный массив фильмов
        self.genres = []           #список жанров
        self.tags_corpus = []      #"мешок"/"corpus" пользовательских тегов
        self.keywords_corpus = []  #"мешок"/"corpus" ключевых слов
        self.overview_corpus = []  #"мешок"/"corpus" overview
        self.actors_corpus = []    #"мешок"/"corpus" actors
        self.crew_corpus = []      #"мешок"/"corpus" crew
    pass
    
    def year_extraction(self):
        """ извлекает год выпуска фильма и добавляет его в массив,
            возвращает массив """
        def year_extract(column):
            try:
                year = int(column.replace(' ','')[-5:-1])
            except ValueError:
                year = 0
            pass
            return year
        
        self.movies['year'] = self.movies['title'].apply(year_extract)
        return self.movies
    pass
    
    def avg_users_rating(self, df_ratings):
        """ считает средний пользовательский рейтинг фильма и добавляет его в массив,
            возвращает массив """
        def avg_rating(column):
            avg_rating = df_ratings[df_ratings['movieId'] == column]['rating'].mean()
            if pd.isna(avg_rating):
                return 0
            else: 
                return avg_rating
            pass
    
        self.movies['avg_rating'] = self.movies['movieId'].apply(avg_rating)
        return self.movies
    pass
    
    def rebuild_genre_list(self):
        """ получает список жанров фильмов, сохраняет в переменную класса "genres",
            возвращает список жанров """
        def genre_list(column):
            genres_split = column.split('|')
            if not self.genres:
                self.genres.append(genres_split[0])
            pass
            for genre in genres_split:
                if genre not in self.genres:
                    self.genres.append(genre)
                pass
            pass

        _ = self.movies['genres'].apply(genre_list)
        self.genres.sort()
        return self.genres
    pass
        
    def genre_vectorization(self, rebuild_genre_list = False):
        """ для каждого фильма получает вектор жанров и добавляет его в массив,
            возвращает массив """
        #если список жанров пуст или нужно его перестроить
        if not self.genres or rebuild_genre_list:
            _ = self.rebuild_genre_list()
        pass
        #получаем векторы жанров для каждого фильма
        def genre_vect(column):
            genre_vect = []
            genres_split = column.split('|')
            for genre in self.genres:
                if genre in genres_split:
                    genre_vect.append(1) #'1' в позиции, если фильм отмечен жанром
                else:
                    genre_vect.append(0) #'0' в позиции, если фильм НЕ отмечен жанром
                pass
            pass
            return genre_vect
    
        self.movies['genres_vect'] = self.movies['genres'].apply(genre_vect)
        return self.movies
    pass
 
    def tags_vectorization(self, df_tags):
        """ для каждого фильма получает вектор пользовательских тегов и добавляет его в массив,
            возвращает массив """
        
        def tag_prep(message, stop_words, morph):
            if str(message) == 'nan':
                message = ''
            message_ = re.sub(r"[^A-Za-z ]"," ", message)                       #оставляем только буквы
            tokens = word_tokenize(message_)                                    #разбиваем на слова
            tokens = [i for i in tokens if (i not in stop_words)]               #исключаем стопслова
            tokens = list(map(lambda x: morph.parse(x)[0].normal_form, tokens)) #приводим к нормальной форме
            return tokens
    
        morph = pymorphy2.MorphAnalyzer() #класс для получения нормальной формы (инфинитива) слова
        stop_words = stopwords.words('english') #подключаем стопслова

        #получаем список токенов
        tokens_list = df_tags['tag'].apply(tag_prep, stop_words = stop_words, morph = morph)

        #собираем "мешок слов"/"корпус" из токенов (текстов) сообщений
        self.tags_corpus = [" ".join(tokens) for tokens in tokens_list]

        #получаем массив векторов пользовательских тегов
        vectors = CountVectorizer().fit_transform(self.tags_corpus).toarray()

        #зафиксируем суммарный вектор тегов фильма
        def vect_summ(column):   
            vect_summ = np.zeros(vectors.shape[1], dtype = int)       #нулевой вектор
            for index in df_tags[df_tags['movieId'] == column].index: #суммируем векторы тегов фильма
                vect_summ = vect_summ | vectors[index]
            pass
            return vect_summ

        self.movies['tags_vect'] = self.movies['movieId'].apply(vect_summ)
        return self.movies
    pass
        
    def req_GetKeywords(self, api_key, df_links):
        """ для каждого фильма получает ключевые слова, сохраняет в файл, создает вектор ключевых слов,
            добавляет его в массив, возвращает массив """
            
        with open('req_GetKeywords.csv', "w", newline='\n') as csv_file:
            writer = csv.writer(csv_file, delimiter=',')
            writer.writerow(['movieId', 'keywords'])
            
        for item in df_links.index:
            movieId = df_links.at[item, 'movieId']
            if not np.isnan(df_links.at[item, 'tmdbId']):
                tmdbId = int(df_links.at[item, 'tmdbId'])
                #получаем инфу по очередному фильму
                res = rq.get(f'https://api.themoviedb.org/3/movie/{tmdbId}/keywords?api_key={api_key}')
                if res.status_code == 200: #если есть ответ
                    keywords = pd.DataFrame.from_dict(res.json()['keywords'])
                    if len(keywords) > 0:
                        keywords_str = "|".join(keywords['name'])
                    pass
                    #записываем все в файл
                    with open('req_GetKeywords.csv', "a", newline='\n') as csv_file:
                        writer = csv.writer(csv_file, delimiter=',')
                        writer.writerow([movieId, keywords_str])
                    pass
                pass
            #время задержки итерации для предотвращения блокировки ресурсом
            #time.sleep(0.2)
            #вывод прогресса
            clear_output(wait = True)
            print(f'Прогресс: {item + 1} / {df_links.shape[0]}.')
        pass
    pass

    def add_Keywords(self):
        try:
            #векторизация ключевых слов
            keywords = pd.read_csv('req_GetKeywords.csv', ',')
    
            def keywords_prep(message, stop_words, morph):
                if str(message) == 'nan':
                    message = ''
                message_ = message.split('|')
                message_ = re.sub(r"[^A-Za-z ]"," ", message)                       #оставляем только буквы
                tokens = word_tokenize(message_)                                    #разбиваем на слова
                tokens = [i for i in tokens if (i not in stop_words)]               #исключаем стопслова
                tokens = list(map(lambda x: morph.parse(x)[0].normal_form, tokens)) #приводим к нормальной форме
                return tokens
            pass
    
            morph = pymorphy2.MorphAnalyzer() #класс для получения нормальной формы (инфинитива) слова
            stop_words = stopwords.words('english') #подключаем стопслова

            #получаем список токенов
            keywords['tokens'] = keywords['keywords'].apply(keywords_prep, stop_words = stop_words, morph = morph)

            #собираем "мешок слов"/"корпус" из токенов (текстов) сообщений
            self.keywords_corpus = [" ".join(tokens) for tokens in keywords['tokens']]

            #получаем массив векторов пользовательских тегов
            vectors = CountVectorizer().fit_transform(self.keywords_corpus).toarray()
            keywords['keywords_vect'] = [vectors[i] for i in range(0, len(vectors))]
    
            #фиксируем в массиве фильмов вектор ключевых слов фильма, если есть ключевые слова, 
            #либо нулевой вектор
            def keywords_vect(column):   
                vect_summ = np.zeros(vectors.shape[1], dtype = int)       #нулевой вектор
                for index in keywords[keywords['movieId'] == column].index: #суммируем векторы тегов фильма
                    vect_summ = vect_summ | vectors[index]
                pass
                return vect_summ
            pass

            self.movies['keywords_vect'] = self.movies['movieId'].apply(keywords_vect)
            return self.movies
        
        except FileNotFoundError:
            print('Не найден файл: req_GetKeywords.csv')
    pass

    def req_GetOverview(self, api_key, df_links):
        """ для каждого фильма получает overview, сохраняет в файл, создает вектор текста overview,
            добавляет его в массив, возвращает массив """
            
        with open('req_GetOverview.csv', "w", newline='\n') as csv_file:
            writer = csv.writer(csv_file, delimiter=',')
            writer.writerow(['movieId', 'overview'])
            
        for item in df_links.index:
            movieId = df_links.at[item, 'movieId']
            if not np.isnan(df_links.at[item, 'tmdbId']):
                tmdbId = int(df_links.at[item, 'tmdbId'])
                #получаем инфу по очередному фильму
                res = rq.get(f'https://api.themoviedb.org/3/movie/{tmdbId}?api_key={api_key}&language=en-US')
                if res.status_code == 200: #если есть ответ, записываем все в файл
                    with open('req_GetOverview.csv', "a", newline='\n') as csv_file:
                        writer = csv.writer(csv_file, delimiter=',')
                        writer.writerow([movieId, res.json()['overview']])
                    pass
                pass
            #время задержки итерации для предотвращения блокировки ресурсом
            #time.sleep(0.2)
            #вывод прогресса
            clear_output(wait = True)
            print(f'Прогресс: {item + 1} / {df_links.shape[0]}.')
        pass
    pass

    def add_Overviews(self):
        try:
            #векторизация ключевых слов
            overview = pd.read_csv('req_GetOverview.csv', ',')
    
            def keywords_prep(message, stop_words, morph):
                if str(message) == 'nan':
                    message = ''
                message_ = re.sub(r"[^A-Za-z ]"," ", message)                       #оставляем только буквы
                tokens = word_tokenize(message_)                                    #разбиваем на слова
                tokens = [i for i in tokens if (i not in stop_words)]               #исключаем стопслова
                tokens = list(map(lambda x: morph.parse(x)[0].normal_form, tokens)) #приводим к нормальной форме
                return tokens
            pass
    
            morph = pymorphy2.MorphAnalyzer() #класс для получения нормальной формы (инфинитива) слова
            stop_words = stopwords.words('english') #подключаем стопслова

            #получаем список токенов
            overview['tokens'] = overview['overview'].apply(keywords_prep, stop_words = stop_words, morph = morph)

            #собираем "мешок слов"/"корпус" из токенов (текстов) сообщений
            self.overview_corpus = [" ".join(tokens) for tokens in overview['tokens']]

            #получаем массив векторов пользовательских тегов
            vectors = CountVectorizer().fit_transform(self.overview_corpus).toarray()
            overview['overview_vect'] = [vectors[i] for i in range(0, len(vectors))]
    
            #фиксируем в массиве фильмов вектор ключевых слов фильма, если есть ключевые слова, 
            #либо нулевой вектор
            def overview_vect(column):   
                vect_summ = np.zeros(vectors.shape[1], dtype = int)       #нулевой вектор
                for index in overview[overview['movieId'] == column].index: #суммируем векторы тегов фильма
                    vect_summ = vect_summ | vectors[index]
                pass
                return vect_summ
            pass

            self.movies['overview_vect'] = self.movies['movieId'].apply(overview_vect)
            return self.movies
        
        except FileNotFoundError:
            print('Не найден файл: req_GetOverview.csv')
    pass
    
    def req_GetPlayLists(self, api_key, df_links):
        """ для каждого фильма получает количество пользовательских play_lists, сохраняет в файл, 
            возвращает массив """
     
        with open('req_GetPlayLists.csv', "w", newline='\n') as csv_file:
            writer = csv.writer(csv_file, delimiter=',')
            writer.writerow(['movieId', 'playlists'])
            
        for item in df_links.index:
            movieId = df_links.at[item, 'movieId']
            if not np.isnan(df_links.at[item, 'tmdbId']):
                tmdbId = int(df_links.at[item, 'tmdbId'])
                #получаем инфу по очередному фильму
                res = rq.get(f'https://api.themoviedb.org/3/movie/{tmdbId}/lists?api_key={api_key}&language=en-US&page=1')
                if res.status_code == 200: #если есть ответ, записываем все в файл
                    with open('req_GetPlayLists.csv', "a", newline='\n') as csv_file:
                        writer = csv.writer(csv_file, delimiter=',')
                        writer.writerow([movieId, res.json()['total_results']])
                pass
            pass
            
            #time.sleep(0.2) #время задержки итерации для предотвращения блокировки ресурсом
            clear_output(wait = True) #вывод прогресса
            print(f'Прогресс: {item + 1} / {df_links.shape[0]}.')
        pass
    pass
      
    def add_Playlists(self):
        try:
            play_lists = pd.read_csv('req_GetPlayLists.csv', ',')
       
            def playlists_set(column):   
                total_playlists = 0
                for index in play_lists[play_lists['movieId'] == column].index: #суммируем playlists фильма
                    total_playlists += play_lists.at[index, 'playlists']
                pass
                return total_playlists
            pass
        
            self.movies['playlists'] = self.movies['movieId'].apply(playlists_set)
            return self.movies
        
        except FileNotFoundError:
            print('Не найден файл: req_GetPlayLists.csv')
    pass
        
    def req_GetDetails(self, api_key, df_links):
        """ для каждого фильма получает некоторые его характеристики, сохраняет в файл, 
            возвращает массив """
            
        new_columns = ['adult', 'budget', 'popularity', 'revenue', 'vote_average', 'vote_count']

        with open('req_GetDetails.csv', "w", newline='\n') as csv_file:
            writer = csv.writer(csv_file, delimiter=',')
            writer.writerow(new_columns)
            
        for item in df_links.index:
            movieId = df_links.at[item, 'movieId']
            if not np.isnan(df_links.at[item, 'tmdbId']):
                tmdbId = int(df_links.at[item, 'tmdbId'])
                #получаем инфу по очередному фильму
                res = rq.get(f'https://api.themoviedb.org/3/movie/{tmdbId}?api_key={api_key}&language=en-US')
                if res.status_code == 200: #если есть ответ, записываем все в файл
                    with open('req_GetDetails.csv', "a", newline='\n') as csv_file:
                        writer = csv.writer(csv_file, delimiter=',')
                        writer.writerow([movieId, \
                                         res.json()['adult'], \
                                         res.json()['budget'], \
                                         res.json()['popularity'], \
                                         res.json()['revenue'], \
                                         res.json()['vote_average'], \
                                         res.json()['vote_count']])
                    pass
                pass
            pass
            
            #time.sleep(0.2) #время задержки итерации для предотвращения блокировки ресурсом
            clear_output(wait = True) #вывод прогресса
            print(f'Прогресс: {item + 1} / {df_links.shape[0]}.')
        pass
    pass

    def add_Details(self):
        try:
            new_columns = ['adult', 'budget', 'popularity', 'revenue', 'vote_average', 'vote_count']
            default_values = [False, 0.0, 0.0, 0.0, 0.0, 0]
            
            #заполняем массив movies новыми признаками
            details = pd.read_csv('req_GetDetails.csv', ',')
        
            def details_set(row):
                if len(details[details['movieId'] == row['movieId']].index) > 0:
                    index = details[details['movieId'] == row['movieId']].index[0]
                    for column in new_columns:
                        row[column] = details.at[index, column]
                    pass
                else:
                    for ind, column in enumerate(new_columns):
                        row[column] = default_values[ind]
                    pass
                pass
                return row
            pass
        
            self.movies = self.movies.apply(details_set, axis = 1)
            return self.movies
        
        except FileNotFoundError:
            print('Не найден файл: req_GetDetails.csv')
    pass
     
    def req_GetCredits(self, api_key, df_links):
        """ для каждого фильма получает список актеров и съемочной группы и их рейтинг, 
            сохраняет в файл, получает векторы списков и добавляет их в массив, возвращает массив """
            
        with open('req_GetCredits.csv', "w", newline='\n') as csv_file:
            writer = csv.writer(csv_file, delimiter=',')
            writer.writerow(['movieId', 'actors', 'sum_act_pop', 'crew', 'sum_crew_pop'])
            
        for item in df_links.index: #[8613 : df_links.index.max() + 1]:
            movieId = df_links.at[item, 'movieId']
            if not np.isnan(df_links.at[item, 'tmdbId']):
                tmdbId = int(df_links.at[item, 'tmdbId'])
                #получаем инфу по очередному фильму
                res = rq.get(f'https://api.themoviedb.org/3/movie/{tmdbId}/credits?api_key={api_key}&language=en-US')
                if res.status_code == 200: #если есть ответ
                    #актеры
                    cast = pd.DataFrame.from_dict(res.json()['cast'])
                    if len(cast) > 0:
                        sum_act_pop = cast['popularity'].sum()
                        cast_sort = cast.sort_values(by = 'popularity', ascending = False)
                        actors = "|".join(cast_sort['original_name']) #[: min(5, len(cast_sort))])
                    else:
                        sum_act_pop = 0.0
                        actors = 'none'
                    pass
                    crew = pd.DataFrame.from_dict(res.json()['crew'])
                    if len(crew) > 0:
                        sum_crew_pop = crew['popularity'].sum()
                        crew_sort = crew.sort_values(by = 'popularity', ascending = False)
                        crews = "|".join(crew_sort['original_name']) #[: min(5, len(crew_sort))])
                    else:
                        sum_crew_pop = 0.0
                        crews = 'none'
                    pass
                    #записываем все в файл
                    with open('req_GetCredits.csv', "a", newline='\n') as csv_file:
                        writer = csv.writer(csv_file, delimiter=',')
                        writer.writerow([movieId, \
                                         actors, \
                                         sum_act_pop, \
                                         crews, \
                                         sum_crew_pop])
                    pass
                pass
            pass
            
            #time.sleep(0.2) #время задержки итерации для предотвращения блокировки ресурсом
            
            clear_output(wait = True) #вывод прогресса
            print(f'Прогресс: {item + 1} / {df_links.shape[0]}.')
        pass
    pass
     
    def add_Credits(self):
        try:
            #заполняем массив movies новыми признаками
            credits = pd.read_csv('req_GetCredits.csv', ',')
        
            #векторизация списка актеров и списка съемочной группы
            def fio_prep(actors):
                tokens = []
                fios = actors.split('|') #разбиваем ФИО
                for fio in fios:
                    token = word_tokenize(fio) #разбиваем на слова
                    tokens.extend(token)   
                return tokens #возвращаем токен
            pass
         
            #получаем список токенов
            credits['act_tokens'] = credits['actors'].apply(fio_prep)
            credits['crew_tokens'] = credits['crew'].apply(fio_prep)

            #собираем "мешок слов"/"корпус" из токенов (текстов) сообщений
            self.actors_corpus = [" ".join(tokens) for tokens in credits['act_tokens']]
            self.crew_corpus = [" ".join(tokens) for tokens in credits['crew_tokens']]

            #получаем массив векторов списка актеров и векторов списка съемочной группы
            act_vectors = CountVectorizer().fit_transform(self.actors_corpus).toarray()
            crew_vectors = CountVectorizer().fit_transform(self.crew_corpus).toarray()
    
            #фиксируем в массиве фильмов вектор актеров, если есть список актеров, либо нулевой вектор
            def act_crew_vect(row):   
                if len(credits[credits['movieId'] == row['movieId']].index) > 0:
                    index = credits[credits['movieId'] == row['movieId']].index[0] #если есть инф. о фильме
                    row['act_vect'] = act_vectors[index]
                    row['crew_vect'] = crew_vectors[index]
                    row['sum_act_pop'] = credits.at[index, 'sum_act_pop']
                    row['sum_crew_pop'] = credits.at[index, 'sum_crew_pop']
                else:
                    row['act_vect'] = np.zeros(act_vectors.shape[1], dtype = int)   #если нет инф. о фильме
                    row['crew_vect'] = np.zeros(crew_vectors.shape[1], dtype = int)
                    row['sum_act_pop'] = 0.0
                    row['sum_crew_pop'] = 0.0
                pass
                return row
            pass

            self.movies = self.movies.apply(act_crew_vect, axis = 1)
            return self.movies
        
        except FileNotFoundError:
            print('Не найден файл: req_GetCredits.csv')
    pass

In [ ]:
#не выполняется, если запущен, как "модуль"
if __name__ == "__main__":
    pass